In [1]:
import pandas as pd
import numpy as np
import time
import datetime
from tqdm import tqdm_notebook
from collections import Counter

---

## Генерация и предобработка данных

In [ ]:
# Данные закрыты, поэтому прикрепить их не выйдет
locations = pd.read_csv("csv_data\locations.csv") # данные с положением людей в пространстве
infusr = pd.read_csv("csv_data\infusr.csv")       # данные с временем обращения

In [ ]:
# Разбитие на дни
time_arr = np.array([1583269200.0 + i * 86400.0 for i in range(92)])

### Добавим поля со временем:

In [ ]:
time = locations['time']
new_time = []

for t in tqdm_notebook(list(time)):
    day = np.where(time_arr<t)[0]
    if len(day) == 0 or len(day) == len(time_arr):
        new_time.append(-1)
        continue
    new_time.append(day[-1]+1)
    
locations['epoch'] = new_time

In [ ]:
time = infusr['time']
new_time_infusr = []

for t in tqdm_notebook(list(time)):
    day = np.where(time_arr<t)[0]
    if len(day) == 0 or len(day) == len(time_arr):
        new_time_infusr.append(-1)
        continue
    new_time_infusr.append(day[-1]+1)
    
infusr['epoch'] = new_time_infusr

### Проведем отбор интересующих нас людей

In [ ]:
interesting_people1 = set(locations[locations['epoch']==40]['user_id']) |\
                      set(locations[locations['epoch']==41]['user_id'])

In [ ]:
count_day_all = Counter({})
    
for day in tqdm_notebook(range(40,91)):
    day_row = list(set(list(locations[locations['epoch']==day]['user_id'])))
    count_day_all.update(day_row)

In [ ]:
count_day_inf = Counter({})
INF_ID = infusr['user_id']
df = locations[locations['user_id'].isin(set(INF_ID))]
    
for day in tqdm_notebook(range(40,91)):
    day_row = set(list(df[df['epoch']==day]['user_id']))
    count_day_inf.update(day_row)

In [ ]:
interesting_people3 = []

for i in tqdm_notebook(count_day_inf.keys()):
    if count_day_inf[i] > 3:
        interesting_people3.append(i)
        
interesting_people3 = set(interesting_people3)

In [ ]:
all_day_1 = {i:Counter() for i in range(40,91)}
all_day_2 = Counter()

for i in tqdm_notebook(range(40,91)):
    df = list(locations[locations['epoch'] == i]['user_id'])
    all_day_1[i].update(df)
    all_day_2.update(df)

In [ ]:
interesting_people4 = set(np.array(list(all_day_2.keys()))[np.where(np.array(list(all_day_2.values()))>5000)[0]])

In [ ]:
interesting_people = (interesting_people2 | interesting_people3) & interesting_people4

### Для отобранных пользователей запустим генерацию матриц смежности

Матрица будет говорить о том, сколько есть сигналов в одном дне, что два пользователя находились достаточно близко дргу к другом.

In [ ]:
LEFT,RIGHT = 40,91
LOC = locations[locations['epoch'].isin([i for i in range(LEFT,RIGHT)])][locations['user_id'].isin(interesting_people)]

for i in tqdm_notebook(range(LEFT, RIGHT)):
    conj_matrix = np.zeros((len(interesting_people),len(interesting_people)))
    
    X = list(LOC[LOC['epoch']==i]['x'])
    Y = list(LOC[LOC['epoch']==i]['y'])
    USER_ID = list(LOC[LOC['epoch']==i]['user_id'])
    LENGTH = len(USER_ID)

    for ind_ip in tqdm_notebook(range(LENGTH)):
        x_ip = X[ind_ip]
        y_ip = Y[ind_ip]
        cur_ip_ind = id_to_ind[USER_ID[ind_ip]]
        
        for ind_contact in range(ind_ip, LENGTH):

            x_contact, y_contact = X[ind_contact], Y[ind_contact]
            
            if np.sqrt((x_ip-x_contact)**2+(y_ip-y_contact)**2) < 1.5:
                
                conj_matrix[cur_ip_ind][id_to_ind[USER_ID[ind_contact]]] += 1
                
    np.save(f"Matrix_adj/{i}_day.npy",conj_matrix)

---

## Эксперимент  
### План:

1. Создание графов nx-графов для сгенерированных матриц смежности;
2. На полученных графах просимулировать работу модели SIER;
3. Исследовать возможности модели:
    * Выкидываем 10%-50% пользователей из отслеживаемых моделью и исследуем как падает качество предсказания;
    * Выкидываем топ-k людей в лучшими центральностями и исследуем как падает качество предсказания: 
        * исследовать зависимость от k;
        * исследовать зависимость от вида центральности;


In [ ]:
#TODO: transfer from dev branch